## Ejemplo: red neuronal de cero

Este ejemplo consiste en programar una red neuronal directamente en numpy, es decir, no estamos utilizando los mecanismos de diferenciación automática de PyTorch, sino que estamos calculando todo a mano. Es un proyecto interesante para ver el mecanismo interno de entrenamiento de una red.

In [4]:
import numpy as np

class NeuralNetwork:
    def __init__(self):
        # Inicializar los pesos a valores aleatorios
        self.weights1 = np.random.rand(2, 2)
        self.weights2 = np.random.rand(2, 1)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def feedforward(self, X):
        # Calcular la salida de la primera capa
        self.hidden_layer = self.sigmoid(np.dot(X, self.weights1))
        # Calculate la salida de la capa de salida
        self.output_layer = self.sigmoid(np.dot(self.hidden_layer, self.weights2))

        return self.output_layer

    def backpropagation(self, X, y, output,learning_rate):
        # Calcular el error para la capa de salida
        output_error = y-output
        output_delta = output_error * self.sigmoid_derivative(output)

        # Calcular el error para la capa oculta
        hidden_error = output_delta.dot(self.weights2.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_layer)

        # Actualizar los pesos
        self.weights2 += self.hidden_layer.T.dot(output_delta)*learning_rate
        self.weights1 += X.T.dot(hidden_delta)*learning_rate
        return output_error

    def train(self, X, y, learning_rate, epochs):
        for i in range(epochs):
            train_error = self.train_epoch(X,y,learning_rate)
            if (i%1000)==0:
                print("Error de entrenamiento en época=%d %f" % (i, np.mean(train_error)))

    def train_epoch(self, X, y, learning_rate):
        output = self.feedforward(X)
        return self.backpropagation(X, y, output, learning_rate)

    def predict(self, X):
        return self.feedforward(X)


In [5]:
def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-7 # Evitamos la división por cero
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon) # Hacemos clip para evitar log(0)
    return - y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred)


In [6]:
np.random.seed(2032)

# Generamos datos de ejemplo (problema xor)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

# Creamos una instancia de nuestra red neuronal
nn = NeuralNetwork()

# Salida inicial con los pesos altearorios
predictions = nn.predict(X)
print("Predictions with no training ",predictions)
print("Cross-entropy loss", binary_cross_entropy(y, predictions))

# Entrenamos la red por 5000 épocas
nn.train(X, y, learning_rate=0.8, epochs=5000)

# Hacemos las predicciones para los datos de entrenamiento
predictions = nn.predict(X)

for i in range(len(X)):
    print(X[i], "prediction",predictions[i])


Predictions with no training  [[0.60096504]
 [0.6199929 ]
 [0.64297382]
 [0.65644404]]
Cross-entropy loss [[0.91870626]
 [0.47804725]
 [0.44165127]
 [1.06840528]]
Error de entrenamiento en época=0 -0.130094
Error de entrenamiento en época=1000 -0.021381
Error de entrenamiento en época=2000 -0.003837
Error de entrenamiento en época=3000 0.001253
Error de entrenamiento en época=4000 0.002691
[0 0] prediction [0.0722281]
[0 1] prediction [0.87399425]
[1 0] prediction [0.87399335]
[1 1] prediction [0.16688778]
